In [1]:
import pandas as pd
import s3fs
import numpy as np
import swifter

In [2]:
def f(lang, embeddings, index):
    if index % 1000000  == 0:
        print(index)
    return [f'{lookup[lang]+i}:{e}' for i, e in enumerate(embeddings)]

In [3]:
new_names = list(range(1,21))
new_names = list(map(str, new_names))

In [4]:
all_features = ["language"]
test = pd.read_csv("s3://recsys-challenge-2020/val_26_04_2020.tsv", encoding="utf-8",
                    names = all_features, usecols= [7], sep="\x01"
                   )
print('test language loaded')

test language loaded


In [5]:
language_code_name_pairs = pd.read_csv('s3://recsys-challenge-2020/language_code_name_pairs.csv')

In [6]:
test_input = pd.merge(test, language_code_name_pairs, how = 'inner',\
left_on = 'language', right_on = 'code')[['name']]

In [7]:
all_lang_output_test = pd.read_csv('s3://recsys-challenge-2020/all_lang_output_test_26_04_2020.csv')
lang_embedding_test = pd.concat([all_lang_output_test[['sentence_embedding']], test_input[['name']]], axis = 1)
lang_embedding_test.sentence_embedding = lang_embedding_test.sentence_embedding.apply(lambda x: np.fromstring(x[1:-1], sep=' '))
lang_embedding_test = lang_embedding_test.rename(columns={"name": "language"})

In [8]:
lookup = {'ja': 43051816,
 'en': 43051836,
 'pt': 43051856,
 'ko': 43051876,
 'id': 43051896,
 'ar': 43051916,
 'tr': 43051936,
 'it': 43051956,
 'fr': 43051976,
 'ru': 43051996,
 'af': 43052016,
 'hr': 43052036,
 'de': 43052056,
 'et': 43052076,
 'vi': 43052096,
 'tl': 43052116,
 'he': 43052136,
 'fa': 43052156,
 'el': 43052176,
 'ur': 43052196,
 'ta': 43052216,
 'lv': 43052236,
 'fi': 43052256,
 'mk': 43052276,
 'hu': 43052296,
 'ca': 43052316,
 'kn': 43052336,
 'ml': 43052356,
 'pa': 43052376,
 'bn': 43052396,
 'mr': 43052416,
 'te': 43052436,
 'gu': 43052456,
 'bg': 43052476,
 'sw': 43052496}

In [9]:
lang_embedding_test[new_names] = lang_embedding_test.swifter.apply(lambda x: f(x.language, x.sentence_embedding, x.name), axis=1, result_type='expand')

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000



In [10]:
lang_embedding_test.drop(['sentence_embedding', 'language'], axis=1).to_csv('lang_embedd_test.csv',\
                                                                           header = False, index = False, sep = ' ')